In [1]:
import sys
sys.path.insert(0, '../../../fastshap_tf/')
from fastshap import ResizeMask

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import pickle
import numpy as np
import shap
from tqdm.notebook import tqdm
import time

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

from datetime import datetime
import os

In [4]:
from tensorflow.keras.layers import (Input, Layer, Dense, Lambda, 
                                     Dropout, Multiply, BatchNormalization, 
                                     Reshape, Concatenate, Conv2D, Permute)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Resizing

from tensorflow.keras.datasets import cifar10

In [5]:
#Select GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

## Load Data

In [6]:
BATCH_SIZE = 32
EPOCHS = 100
LR = 1e-3
INPUT_SHAPE = (224, 224, 3)

### Load Data

In [7]:
from sklearn.model_selection import train_test_split

num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, train_size=0.5, random_state=420)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
#Resize to 224x224

print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'val samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Make TF Dataset
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

50000 train samples
5000 val samples
5000 test samples


### Batch Data

In [8]:
def batch_data(dataset, fn, batch_size=32):
    dataset = dataset.map(fn)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

### Reformat Data

In [9]:
def reformat(x, y):
    
    x = tf.cast(x, tf.float32)
    x = Resizing(INPUT_SHAPE[0], INPUT_SHAPE[1], interpolation='nearest')(x)
    x = tf.keras.applications.resnet50.preprocess_input(x)
    
    return (x, y)

ds_train = batch_data(ds_train, reformat, BATCH_SIZE)
ds_val = batch_data(ds_val, reformat, BATCH_SIZE)
ds_test = batch_data(ds_test, reformat, BATCH_SIZE)

## Load Model

In [10]:
from tensorflow.keras.applications.resnet50 import ResNet50

base_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
)
base_model.trainable = True

model_input = Input(shape=INPUT_SHAPE, name='input')

net = base_model(model_input)
out = Dense(10, activation='softmax')(net)

model = Model(model_input, out)

model_weights_path = 'model/20210514_14_39_21/model_weights.h5'

model.load_weights(model_weights_path)
model.trainable = False

# Train Evaluator

### Prepare Dataset w/ Model Predictions

In [11]:
@tf.function
def make_prediction_data(x, y):
    with tf.device("gpu:2"):
        y_model = model(x)
    
    return (x, y_model)

with tf.device("gpu:2"):
    ds_train_pred = ds_train.map(make_prediction_data)
    ds_val_pred = ds_val.map(make_prediction_data)
    ds_test_pred = ds_test.map(make_prediction_data)

### Save Dir

In [12]:
save_dir = 'evaluator'
model_dir = os.path.join(os.getcwd(), save_dir)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

### Initialize Model

In [13]:
class Random_Bernoulli_Sampler(Layer):
    '''
    Layer to Sample r
    '''
    def __init__(self, num_features, **kwargs):
        super(Random_Bernoulli_Sampler, self).__init__(**kwargs)
        
        self.num_features = num_features
        
        
    def call(self,  logits):
        batch_size = tf.shape(logits)[0]
        
        u = tf.random.uniform(shape=(batch_size, self.num_features),
                                    minval=np.finfo(
                                        tf.float32.as_numpy_dtype).tiny,
                                    maxval=1.0) 

        r = K.cast(tf.stop_gradient(u > 0.5), tf.float32)
        
        return r

In [14]:
base_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
)
base_model.trainable = True

model_input = Input(shape=INPUT_SHAPE, dtype='float32', name='input')
R = Random_Bernoulli_Sampler(14*14)(model_input)
R = ResizeMask(in_shape=INPUT_SHAPE, mask_size=14*14)(R)
xr = Multiply()([model_input, R])

net = base_model(xr)
out = Dense(10, activation='softmax')(net)

evaluator_model = Model(model_input, out)

### Train Model

In [15]:
LR = 1e-3

# Metrics
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='accuracy'),
]

# Model Checkpointing
model_weights_path = os.path.join(model_dir, 'model_weights.h5')
checkpoint = ModelCheckpoint(model_weights_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only=True)

# LR Schedule
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=5, 
                             verbose=1, mode='min', cooldown=1, min_lr=1e-3/10)

# Early Stopping 
earlyStop = EarlyStopping(monitor="val_loss", mode="min", patience=20) 

# Compile Model
CALLBACKS = [checkpoint, earlyStop, reduceLR]
OPTIMIZER = tf.keras.optimizers.Adam(LR)

evaluator_model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)

# Train Model
evaluator_model.fit(ds_train,
          epochs = 100,
          validation_data = ds_val,
          callbacks = CALLBACKS)

# Get Checkpointed Model
print(model_weights_path)
evaluator_model.load_weights(model_weights_path)
evaluator_model = Sequential(   
            [l for l in evaluator_model.layers[-2:]]
        )
evaluator_model.trainable = False

Epoch 1/100
1563/1563 [==============================] - ETA: 0s - loss: 1.1018 - auroc: 0.9307 - auprc: 0.6874 - accuracy: 0.6077
Epoch 00001: val_loss improved from inf to 1.05396, saving model to /gpfs/data/paulab/nj594/fast_shap/experiments/images/cifar10/evaluation/evaluator/model_weights.h5
1563/1563 [==============================] - 276s 177ms/step - loss: 1.1018 - auroc: 0.9307 - auprc: 0.6874 - accuracy: 0.6077 - val_loss: 1.0540 - val_auroc: 0.9372 - val_auprc: 0.7145 - val_accuracy: 0.6372
Epoch 2/100
1563/1563 [==============================] - ETA: 0s - loss: 0.7203 - auroc: 0.9694 - auprc: 0.8408 - accuracy: 0.7486
Epoch 00002: val_loss improved from 1.05396 to 0.84839, saving model to /gpfs/data/paulab/nj594/fast_shap/experiments/images/cifar10/evaluation/evaluator/model_weights.h5
1563/1563 [==============================] - 175s 112ms/step - loss: 0.7203 - auroc: 0.9694 - auprc: 0.8408 - accuracy: 0.7486 - val_loss: 0.8484 - val_auroc: 0.9586 - val_auprc: 0.7940 - val

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1563/1563 [==============================] - ETA: 0s - loss: 0.2023 - auroc: 0.9970 - auprc: 0.9810 - accuracy: 0.9276
Epoch 00008: val_loss did not improve from 0.52764
1563/1563 [==============================] - 173s 111ms/step - loss: 0.2023 - auroc: 0.9970 - auprc: 0.9810 - accuracy: 0.9276 - val_loss: 0.5880 - val_auroc: 0.9775 - val_auprc: 0.9029 - val_accuracy: 0.8238
Epoch 9/100
1563/1563 [==============================] - ETA: 0s - loss: 0.1612 - auroc: 0.9980 - auprc: 0.9874 - accuracy: 0.9431
Epoch 00009: val_loss did not improve from 0.52764
1563/1563 [==============================] - 174s 111ms/step - loss: 0.1612 - auroc: 0.9980 - auprc: 0.9874 - accuracy: 0.9431 - val_loss: 0.6555 - val_auroc: 0.9721 - val_auprc: 0.8899 - val_accuracy: 0.8158
Epoch 10/100
 613/1563 [==========>...................] - ETA: 1:39 - loss: 0.1511 - auroc: 0.9981 - auprc: 0.9886 - accuracy: 0.9458

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1563/1563 [==============================] - ETA: 0s - loss: 0.0791 - auroc: 0.9992 - auprc: 0.9959 - accuracy: 0.9727
Epoch 00014: val_loss did not improve from 0.52764
1563/1563 [==============================] - 165s 105ms/step - loss: 0.0791 - auroc: 0.9992 - auprc: 0.9959 - accuracy: 0.9727 - val_loss: 0.6241 - val_auroc: 0.9737 - val_auprc: 0.9080 - val_accuracy: 0.8462
Epoch 15/100
1563/1563 [==============================] - ETA: 0s - loss: 0.0745 - auroc: 0.9993 - auprc: 0.9964 - accuracy: 0.9739
Epoch 00015: val_loss did not improve from 0.52764
1563/1563 [==============================] - 165s 105ms/step - loss: 0.0745 - auroc: 0.9993 - auprc: 0.9964 - accuracy: 0.9739 - val_loss: 0.5998 - val_auroc: 0.9759 - val_auprc: 0.9097 - val_accuracy: 0.8480
Epoch 16/100
1352/1563 [========================>.....] - ETA: 21s - loss: 0.0666 - auroc: 0.9994 - auprc: 0.9969 - accuracy: 0.9772

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1563/1563 [==============================] - ETA: 0s - loss: 0.0482 - auroc: 0.9996 - auprc: 0.9982 - accuracy: 0.9834
Epoch 00021: val_loss did not improve from 0.52764
1563/1563 [==============================] - 165s 105ms/step - loss: 0.0482 - auroc: 0.9996 - auprc: 0.9982 - accuracy: 0.9834 - val_loss: 0.6689 - val_auroc: 0.9707 - val_auprc: 0.9016 - val_accuracy: 0.8446
Epoch 22/100
1563/1563 [==============================] - ETA: 0s - loss: 0.0446 - auroc: 0.9997 - auprc: 0.9985 - accuracy: 0.9843
Epoch 00022: val_loss did not improve from 0.52764

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.0008573750033974647.
1563/1563 [==============================] - 165s 105ms/step - loss: 0.0446 - auroc: 0.9997 - auprc: 0.9985 - accuracy: 0.9843 - val_loss: 0.6696 - val_auroc: 0.9711 - val_auprc: 0.9035 - val_accuracy: 0.8490
Epoch 23/100
  16/1563 [..............................] - ETA: 2:28 - loss: 0.0552 - auroc: 0.9998 - auprc: 0.9984 - accuracy: 0.9746

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1563/1563 [==============================] - ETA: 0s - loss: 0.0336 - auroc: 0.9998 - auprc: 0.9990 - accuracy: 0.9883
Epoch 00027: val_loss did not improve from 0.52764

Epoch 00027: ReduceLROnPlateau reducing learning rate to 0.0008145062311086804.
1563/1563 [==============================] - 165s 105ms/step - loss: 0.0336 - auroc: 0.9998 - auprc: 0.9990 - accuracy: 0.9883 - val_loss: 0.7099 - val_auroc: 0.9693 - val_auprc: 0.9000 - val_accuracy: 0.8518
/gpfs/data/paulab/nj594/fast_shap/experiments/images/cifar10/evaluation/evaluator/model_weights.h5


In [18]:
# Evaluate
evaluator_model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)
evaluator_model.evaluate(ds_test)

157/157 [==============================] - 5s 33ms/step - loss: 0.4717 - auroc: 0.9782 - auprc: 0.9174 - accuracy: 0.8558


[0.47172021865844727,
 0.9782240986824036,
 0.9173834323883057,
 0.8557999730110168]

In [19]:
evaluator_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 0
Non-trainable params: 23,608,202
_________________________________________________________________
